# Get daily Wikipedia pageviews for topic(s)

#### Import Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_stiles as altstiles
import numpy as np

In [3]:
alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('stiles')

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [5]:
today = pd.to_datetime("today").strftime("%Y%m%d00")

---

## Enter two topics to compare

In [6]:
topics = ["Rose"]

In [47]:
dataframes = []

for t in topics:
    df_src = pd.json_normalize(
        pd.read_json(
            f"https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/user/{t}/monthly/2010010100/{today}"
        )["items"]
    ).sort_values("views", ascending=False)
    dataframes.append(df_src)
    src = pd.concat(dataframes)

In [48]:
src["date"] = pd.to_datetime(src["timestamp"].str[:8])

In [49]:
src["subject"] = src["article"].str.replace("_", " ", regex=False)

In [50]:
df = src.drop(["article", "project", "timestamp", "access", "agent"], axis=1)

In [55]:
df.head(10)

,granularity,views,date,subject
2,monthly,132870,2015-09-01,Rose
7,monthly,126277,2016-02-01,Rose
10,monthly,123903,2016-05-01,Rose
11,monthly,123810,2016-06-01,Rose
3,monthly,119753,2015-10-01,Rose
0,monthly,116793,2015-07-01,Rose
4,monthly,112517,2015-11-01,Rose
12,monthly,112365,2016-07-01,Rose
1,monthly,112344,2015-08-01,Rose
15,monthly,112185,2016-10-01,Rose


---

In [54]:
alt.Chart(df).mark_bar().encode(
    x=alt.X("date:T", title=""),
    y=alt.Y("views", title="", axis=alt.Axis(tickCount=4)),
    facet=alt.Facet("subject", columns=2, title=" "),
).configure_legend(orient="top").properties(
    width=1000,
    height=300,
    title=f"Pageviews on Wikipedia",
)

alt.Chart(...)

In [14]:
df.groupby("subject").agg({"views": sum}).reset_index()

,subject,views
0,Rose,8435182
